In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("physician.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'physician.csv'

In [3]:
df.shape

(141624, 11)

In [4]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,316237,16605,109285.0,2138-03-21,2138-03-21 15:02:00,2138-03-21 15:03:05,Physician,Physician Resident Progress Note,21203.0,NaN,Chief Complaint:\n 24 Hour Events:\n Continu...
1,316238,29075,179159.0,2116-02-07,2116-02-07 15:37:00,2116-02-07 15:37:10,Physician,Physician Resident Progress Note,21203.0,NaN,Chief Complaint:\n 24 Hour Events:\n EGD d...
2,316241,29075,179159.0,2116-02-07,2116-02-07 15:37:00,2116-02-07 16:05:26,Physician,Physician Resident Progress Note,21203.0,NaN,24 Hour Events:\n EGD demonstrated no eviden...
3,316242,29075,179159.0,2116-02-07,2116-02-07 15:37:00,2116-02-07 16:08:06,Physician,Physician Resident Progress Note,21203.0,NaN,24 Hour Events:\n EGD demonstrated no eviden...
4,316243,31608,152365.0,2133-01-16,2133-01-16 16:12:00,2133-01-16 16:12:47,Physician,Physician Resident Progress Note,21203.0,NaN,Chief Complaint:\n 24 Hour Events:\n Recei...


In [5]:
df['SUBJECT_ID'].value_counts()

99383    481
55672    464
30202    463
31942    365
27427    363
        ... 
61427      1
77026      1
67648      1
93996      1
72968      1
Name: SUBJECT_ID, Length: 7623, dtype: int64

这里后面以单个subject_id测试了一下各类note的分布情况，以及包含soap的情况

In [6]:
df_1 = df[df['SUBJECT_ID']==99383]
df_1['DESCRIPTION'].value_counts()

Physician Resident Progress Note                      332
Physician Attending Progress Note                     101
Physician Resident/Attending Progress Note - MICU      27
Physician Resident Admission Note                      10
ICU Attending Note                                      4
MICU Attending Admission Note                           1
Attending progress note                                 1
ICU Attending  Note                                     1
Attending Note                                          1
ICU Event Note                                          1
Attending  Note                                         1
Physician Resident/Attending Admission Note - MICU      1
Name: DESCRIPTION, dtype: int64

In [7]:
sorted_df_1 = df_1.sort_values(by='ROW_ID', ascending=False)
sorted_df_1.head(5)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
121666,692782,99383,181682.0,2131-08-22,2131-08-22 09:20:00,2131-08-22 15:31:12,Physician,Physician Attending Progress Note,20401.0,NaN,Chief Complaint: leg hematoma\n I saw and ex...
121626,692754,99383,181682.0,2131-08-22,2131-08-22 09:20:00,2131-08-22 09:20:13,Physician,Physician Attending Progress Note,21242.0,NaN,Chief Complaint: leg hematoma\n I saw and ex...
121687,692752,99383,181682.0,2131-08-22,2131-08-22 07:14:00,2131-08-22 09:07:25,Physician,Physician Resident Progress Note,18019.0,NaN,Chief Complaint: Right thigh hematoma\n 24 H...
121678,692740,99383,181682.0,2131-08-22,2131-08-22 07:14:00,2131-08-22 07:27:55,Physician,Physician Resident Progress Note,16045.0,NaN,Chief Complaint: Right thigh hematoma\n 24 H...
121676,692738,99383,181682.0,2131-08-22,2131-08-22 07:14:00,2131-08-22 07:16:39,Physician,Physician Resident Progress Note,16045.0,NaN,Chief Complaint: Right thigh hematoma\n 24 H...


In [2]:
def contains_subjective(text):
    subjective_keywords = ["reports", "complains", "symptoms", "history of"]
    return any(keyword in text.lower() for keyword in subjective_keywords)

def contains_objective(text):
    objective_keywords = ["vital signs", "physical examination", "labs", "imaging", "findings"]
    return any(keyword in text.lower() for keyword in objective_keywords)

def contains_assessment(text):
    objective_keywords = ["assessment"]
    return any(keyword in text.lower() for keyword in objective_keywords)

def contains_plan(text):
    objective_keywords = ["plan"]
    return any(keyword in text.lower() for keyword in objective_keywords)

# test
notes = ['Physician Resident Progress Note',
         'Physician Attending Progress Note',
         'Physician Resident/Attending Progress Note - MICU',
         'Physician Resident Admission Note',
         'ICU Attending Note',
         'MICU Attending Admission Note',
         'Attending progress note',
         'ICU Attending  Note',
         'Attending Note',
         'ICU Event Note',
         'Attending  Note',
         'Physician Resident/Attending Admission Note - MICU']
for note in notes:
    text = df_1.loc[df_1['DESCRIPTION']==note, 'TEXT'].values[0]
    print(note)
    if contains_subjective(text):
        print("This note contains subjective information.")
    if contains_objective(text):
        print("This note contains objective information.")
    if contains_assessment(text):
        print("This note contains assessment.")
    if contains_plan(text):
        print("This note contains plan.")

NameError: name 'df_1' is not defined

上面是test，下面是extract and then split assessment and plan，然后存储为json的过程

In [21]:
import pandas as pd
import json
import re

In [11]:
df = pd.read_csv("physician.csv")

In [4]:
def contains_subjective(text):
    subjective_keywords = ["reports", "complains", "symptoms", "history of"]
    return any(keyword in text.lower() for keyword in subjective_keywords)

def contains_objective(text):
    objective_keywords = ["vital signs", "physical examination", "labs", "imaging", "findings"]
    return any(keyword in text.lower() for keyword in objective_keywords)

def contains_assessment(text):
    objective_keywords = ["assessment"]
    return any(keyword in text.lower() for keyword in objective_keywords)

def contains_plan(text):
    objective_keywords = ["plan"]
    return any(keyword in text.lower() for keyword in objective_keywords)

def check_condition(text):
    if contains_subjective(text) and contains_objective(text) and contains_assessment(text) and contains_plan:
        return True
    return False

# 这里的subjective and objective的keywords不确定是否OK

In [5]:
# 直接根据assessment and plan来分割，前面是s_and_o，后面是a_and_p
def split_assessment_plan(text):
    match = re.search(r'assessment and plan', text, re.IGNORECASE)
    
    if match:
        split_index = match.start()
        s_and_o = text[:split_index].strip()
        a_and_p = text[split_index:].strip()
        return s_and_o, a_and_p
    else:
        return None, None

In [6]:
result_list = []

# grouped by SUBJECT_ID
grouped = df.groupby('SUBJECT_ID')

for subject_id, group in grouped:
    # sorted by row_id
    sorted_group = group.sort_values(by='ROW_ID', ascending=False)
    
    for index, row in sorted_group.iterrows():
        if check_condition(row['TEXT']):
            row_data = row.to_dict()
            
            s_and_o, a_and_p = split_assessment_plan(row['TEXT'])
            
            if s_and_o and a_and_p:
                row_data['s_and_o'] = s_and_o
                row_data['a_and_p'] = a_and_p
                result_list.append(row_data)
                break
            else:
                continue
    
print(len(result_list))

# store into a json file
with open('unique_notes.json', 'w') as json_file:
    json.dump(result_list, json_file, indent=4)

print("unique_notes.json is done")

# 先以subject_id分组，然后对其row_id进行从近到远排序，对应的就是降序；然后从上到下遍历，直到取到满足soap condition并且能分割assessment and plan的note
# 这样结束之后有5074条满足

NameError: name 'df' is not defined

In [5]:
import os
import json

with open("data/virtual_hospital/MIMIC_Modified.json", "r") as f:
    data = json.load(f)
    
for note in reversed(data):
    print(note['ROW_ID'])
    print(note['SUBJECT_ID'])
    break

719241
99991


In [9]:
import os
import json

from langchain.chains.llm import LLMChain
from langchain_core.prompts import load_prompt
from langchain_core.tools import StructuredTool
from langchain_openai import ChatOpenAI

from tools import patient_agent, doctor_agent
import tqdm
# loop over all the folders in data_virtual_hospital

gpt4o = "gpt-4o"
max_iter = 30
self_critic = True


def assistant(chat_history: str, patient_info: str):
    """
        Tool that calls a chain to find the missing information,
            valid parameter include "chat_history": chat_history, "patient_info": patient's background
        """
    prompt = load_prompt("prompts/Virtual_hospital/Conversation_self_critic.yaml")

    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)
    chain = LLMChain(llm=llm, prompt=prompt, verbose=False)
    response = chain.invoke({"chat_history": chat_history, "patient_info": patient_info})
    return response["text"]


if __name__ == "__main__":

    # root_dir = "./data/virtual_hospital/MIMIC_Modified2"
    # list = os.listdir(root_dir)
    
    with open("data/virtual_hospital/MIMIC_Modified.json", "r") as f:
        notes = json.load(f) 
    assistant_ = StructuredTool.from_function(assistant)
    
    counter = 0

    for data in reversed(notes):
        ROW_ID = data['ROW_ID']
        SUBJECT_ID = data['SUBJECT_ID']
        output_filename = f"{SUBJECT_ID}_{ROW_ID}.txt"
        
        if os.path.exists(f"output/MIMIC/Conversation_0925/{output_filename}"):
            continue
        # print(f"{len(os.listdir('output/MIMIC/Conversation2'))}/{len(os.listdir(root_dir))}: {file}")
        # with open(f"{root_dir}/{file}") as json_file:
        #     data = json.load(json_file)
        #     json_file.close()

        doctor_info = (f"Patient's Age: {data['Doctor_Actor']['Age']},"
                       f"Patient's Gender: {data['Doctor_Actor']['Gender']},"
                       f"Reason for Visit: {data['Doctor_Actor']['Reason_for_visit']}"
                       f"Physical Examination Findings: {data['Patient_Actor']['Physical_Examination_Findings']}"
                       f"Vital Signs: {data['Patient_Actor']['Vital_Signs']}"
                       f"Review of Systems: {data['Patient_Actor']['Review_of_Systems']} "
                       )
        patient_info = ""
        print(data['Patient_Actor'].keys())
        # for key in data['Patient_Actor'].keys():
        patient_info = "".join(f"Patient's {key}: {data['Patient_Actor'][key]}," for key in data['Patient_Actor'].keys())
        print(patient_info)
        break

dict_keys(['Chief_Complaint', 'HPI', 'Allergies', 'Past_Medical_History', 'Current_Medications', 'Vital_Signs', 'Physical_Examination_Findings', 'Family_History', 'Social_History', 'Review_of_Systems', 'Assessment', 'Plan'])
Patient's Chief_Complaint: Consultation regarding evaluation and management of perioperative risk.,Patient's HPI: 50 y/o woman with hypertension, dyslipidemia, tobacco use, and peripheral artery disease who had recent left external iliac to above knee artery bypass and was admitted with bleeding at the site of an infected groin wound.,Patient's Allergies: Penicillin,Patient's Past_Medical_History: Hypertension, Dyslipidemia, Peripheral arterial disease with history of ischemic ulcerations in right lower extremity and disabling claudication in left lower extremity. Surgeries include right common iliac PTA, right common femoral endarterectomy/patch angioplasty, left femoral to right above knee artery bypass, and left external iliac to above knee artery bypass.,Patien

In [7]:
import pandas as pd
import json
import os

with open("data/virtual_hospital/MIMIC_Modified.json", 'r') as f:
    data = json.load(f)
    
with open("data/virtual_hospital/physician.csv", 'r') as f:
    physician_notes = pd.read_csv(f)


In [17]:
import re
data_update = []
for note in data:
    ROW_ID = int(note['ROW_ID'])
    SUBJECT_ID = note['SUBJECT_ID']
    original_note = physician_notes[physician_notes["ROW_ID"] == ROW_ID]
    # print(type(original_note['TEXT'].iloc[0]))
    
    s_and_o, a_and_p = split_assessment_plan(original_note['TEXT'].iloc[0])
            
    if s_and_o and a_and_p:
        note['s_and_o'] = s_and_o
        note['a_and_p'] = a_and_p
    data_update.append(note)
print(len(data_update))
with open("data/MIMIC_update.jsons", "w") as f:
    json.dump(data_update, f)


5070


In [20]:
with open("data/MIMIC_update.jsons", 'r') as f:
    notes_update = json.load(f)
counter = 0
for note in notes_update:
    counter+=1
print(counter)

5070


In [14]:
import json
import os
from tqdm import tqdm
import pandas as pd

# Initialize an empty DataFrame with columns 'prompts' and 'completion'
df = pd.DataFrame(columns=['instruction', 'output', 'input'])

with open(f"data/virtual_hospital/MIMIC_update_1007.json", 'r') as f:
    physician_notes = json.load(f)

with open("data_process/SFT_dataset_utils/InfoGatherQA_instruction.txt", 'r') as f:
    instruction_template = f.read()
# print(instruction)
# instruction = instruction_template.replace("<info>", "Patient's information").replace("<chat_history>", "How's your feeling?")
# print(instruction)

for filename in tqdm(os.listdir("data/virtual_hospital/conversation"), desc="Processing files"):
    
    # get the conversation data
    with open(f"data/virtual_hospital/conversation/{filename}", 'r', encoding="ISO-8859-1") as f:
        conversation = f.read()

    # get patient's information
    filename = filename[:-4]
    ROW_ID = filename.split("_")[1]
    SUBJECT_ID = filename.split("_")[0]
    id = f"{SUBJECT_ID}_{ROW_ID}"

    patient_info = str(physician_notes[id]['Doctor_Actor'])
    instruction_template = instruction_template.replace("<info>", patient_info)
    iterations = conversation.split("\n")
   

    conversations = ""
    # print(len(iterations))
    for i in range(0, len(iterations), 2):
        # instruction = instruction_template.replace("<chat_history>", conversations)

        if "Doctor" in iterations[i]:
            output = iterations[i]
            new_row = pd.DataFrame([[instruction_template, output.replace("Doctor: ", ""), conversations]], columns=['instruction', 'output', 'input'])
            df = pd.concat([df, new_row], ignore_index=True)
        else: 
            # print(f"break at {i} th iteration Due to missing doctor")
            # print(iterations[i])

            break
        
        if i+1 < len(iterations)-1 and "Doctor" in iterations[i] and "Patient" in iterations[i+1]:
            conversations += f"{iterations[i]}\n"
            conversations += f"{iterations[i+1]}\n"
        else:
            # print(f"break at {i} th iteration")
            break
df.to_csv('data/SFT_dataset/conversation.csv')
    # break




Processing files: 100%|██████████| 5070/5070 [16:49<00:00,  5.02it/s]


In [12]:
import pandas as pd
data_1 = pd.read_csv('data/SFT_dataset/conversation_lite.csv', index_col=0)
print(data_1.shape)
print(data_1.columns)
print(data_1['input'][1])
print(data_1['output'][1])

(30, 3)
Index(['instruction', 'output', 'input'], dtype='object')
Doctor: When did you first notice the changes in your mental status?
Patient: I first noticed the changes in my mental status about a few days ago. It started with feeling more confused and disoriented than usual.

Can you describe how your confusion and disorientation have changed over the past few days?


In [1]:
import json 
with open("data/SFT_dataset/llama_assessment.json", 'r') as f:
    assessments = json.load(f)
with open("data/SFT_dataset/llama_plan1.json", "r") as f:
    plan1  = json.load(f)
with open("data/SFT_dataset/llama_plan2.json", "r") as f:
    plan2 = json.load(f)



In [8]:
import pandas as pd
assessments_df = pd.DataFrame(assessments)
plan1_df = pd.DataFrame(plan1)
plan2_df = pd.DataFrame(plan2)
conversation_df = pd.read_csv("data/SFT_dataset/conversation.csv", index_col=0)

In [9]:
print(conversation_df.shape)
print(assessments_df.shape)
print(plan1_df.shape)
print(plan2_df.shape)

(152078, 3)
(5070, 3)
(4755, 3)
(4755, 3)


In [12]:
conversation_train = conversation_df[: 5000]
assessments_train = assessments_df[: 4000]
plan1_train = plan1_df[: 4000]
plan2_train = plan2_df[: 4000]
train = pd.concat([conversation_train, assessments_train, plan1_train, plan2_train], axis=0)
train.to_csv("data/SFT_dataset/train_01.csv", index=False)

In [10]:
conversation_df.columns

Index(['instruction', 'output', 'input'], dtype='object')

In [36]:
conversation_val = conversation_df[150000: 151000]
assessments_val = assessments_df[4000: 4500]
plan1_val = plan1_df[4000: 4400]
plan2_val = plan2_df[4000: 4400]
val = pd.concat([conversation_val, assessments_val, plan1_val, plan2_val], axis=0)
val.to_csv("data/SFT_dataset/val.csv", index=False)

In [37]:
conversation_test = conversation_df[151000:]
assessments_test = assessments_df[4500:]
plan1_test = plan1_df[4400:]
plan2_test = plan2_df[4400:]
test = pd.concat([conversation_test, assessments_test, plan1_test, plan2_test], axis=0)
test.to_csv("data/SFT_dataset/test.csv", index=False)

In [9]:
print(train.shape)
# print(val.shape)
# print(test.shape)

(27240, 5)


In [38]:
import json
import os
with open(f"data/virtual_hospital/MIMIC_update.json", 'r') as f:
    physician_notes = json.load(f)

print(type(physician_notes))
physician_notes_new = {}
for note in physician_notes:
    id = f"{note['SUBJECT_ID']}_{note['ROW_ID']}"
    physician_notes_new[id] = note
with open(f"data/virtual_hospital/MIMIC_update_1007.json", "w") as f:
    json.dump(physician_notes_new, f, indent=4)
print(type(physician_notes_new))

<class 'list'>
<class 'dict'>


In [42]:
with open(f"data/virtual_hospital/MIMIC_update_1007.json", 'r') as f:
    physician_notes = json.load(f)
print(type(physician_notes))
print(str(physician_notes['109_385223']['Doctor_Actor']))
patient_info = str(physician_notes['109_385223']['Doctor_Actor'])
print(type(patient_info))

<class 'dict'>
{'Age': '25', 'Gender': 'Female', 'Reason_for_visit': 'Dyspnea and weakness'}
<class 'str'>


In [1]:
#### Loading data from MIMIC IV
import pandas as pd

discharge = pd.read_csv("C:\Projects\VirtualHospital\data\MIMIC_IV\discharge.csv")
discharge_detail = pd.read_csv("C:\Projects\VirtualHospital\data\MIMIC_IV\discharge_detail.csv")
# radiology_detail = pd.read_csv("C:\Projects\VirtualHospital\data\MIMIC_IV\radiology_detail.csv")
# radiology = pd.read_csv("C:\Projects\VirtualHospital\data\MIMIC_IV\radiology.csv")

In [18]:
print(discharge.columns)
discharge.iloc[0]
with open("C:/Projects/VirtualHospital/data/MIMIC_IV/discharge0.txt", 'w') as f:
    f.write(discharge.iloc[0]['text'])

Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text'],
      dtype='object')


In [13]:
print(discharge_detail.columns)
discharge_detail.iloc[1]

Index(['note_id', 'subject_id', 'field_name', 'field_value', 'field_ordinal'], dtype='object')


note_id          10000032-DS-22
subject_id             10000032
field_name               author
field_value                 ___
field_ordinal                 1
Name: 1, dtype: object

In [29]:
#### Loading data from MIMIC IV
import pandas as pd

# discharge = pd.read_csv("C:\Projects\VirtualHospital\data\MIMIC_IV\discharge.csv")
# discharge_detail = pd.read_csv("C:\Projects\VirtualHospital\data\MIMIC_IV\discharge_detail.csv")
# radiology_detail = pd.read_csv("C:/Projects/VirtualHospital/data/MIMIC_IV/radiology_detail.csv")
admissions = pd.read_csv("C:/Projects/VirtualHospital/data/MIMIC_IV/admissions.csv", nrows=5)

In [36]:
print(admissions.columns)
# admissions.iloc[0]['text']
output = ""
for key in admissions.columns:
    output += f"{key}: {admissions.iloc[0][key]}\n\n"
with open("C:/Projects/VirtualHospital/data/MIMIC_IV/admissions.txt", 'w') as f:
    f.write(output)

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admit_provider_id', 'admission_location',
       'discharge_location', 'insurance', 'language', 'marital_status', 'race',
       'edregtime', 'edouttime', 'hospital_expire_flag'],
      dtype='object')
